In [1]:
!pip install numpy pandas opencv-python torch torchvision ultralytics pydicom matplotlib

  Using cached matplotlib-3.10.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (11 kB)
  Using cached pillow-11.1.0-cp313-cp313-manylinux_2_28_x86_64.whl.metadata (9.1 kB)
  Using cached contourpy-1.3.1-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (5.4 kB)
  Using cached cycler-0.12.1-py3-none-any.whl.metadata (3.8 kB)
  Using cached fonttools-4.57.0-cp313-cp313-manylinux_2_5_x86_64.manylinux1_x86_64.manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (102 kB)
  Using cached kiwisolver-1.4.8-cp313-cp313-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (6.2 kB)
  Using cached pyparsing-3.2.3-py3-none-any.whl.metadata (5.0 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.0/63.0 MB 3.8 MB/s eta 0:00:00m eta 0:00:010:00:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 766.6/766.6 MB 3.6 MB/s eta 0:00:00m eta 0:00:010:00:06
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.6 MB/s eta 0:00:00m eta 0:00:010:00:03
   ━━━━━━━━━━

In [2]:
import pandas as pd
import json

In [18]:
# Load the Excel file
xls_file = "/home/yukta/devel/wip/AIML/ultrasound-analysis/data/annotations/ObjectDetection.xlsx"
df = pd.read_excel(xls_file)
print(df.head())  # Show the first few rows to verify the structure


     fname   structure  h_min  w_min  h_max  w_max
0  168.png     thalami    178    171    244    261
1  168.png  nasal bone     96    308    111    349
2  168.png      palate    133    300    205    408
3  168.png  nasal skin     86    324     95    349
4  168.png   nasal tip     79    345     89    376


In [19]:
import json
import os

In [26]:
# Check if required columns exist
required_columns = {"fname", "structure", "h_min", "w_min", "h_max", "w_max"}
if not required_columns.issubset(df.columns):
    raise KeyError(f"Missing columns in Excel file: {required_columns - set(df.columns)}")

In [20]:
# COCO format initialization
coco_data = {
    "images": [],
    "annotations": [],
    "categories": []
}

In [27]:
# Define category mapping
category_mapping = {structure: idx for idx, structure in enumerate(df["structure"].unique(), 1)}
for structure, idx in category_mapping.items():
    coco_data["categories"].append({"id": idx, "name": structure})

In [21]:
category_map = {}
category_id = 1
image_id_map = {}
image_id = 1
annotation_id = 1

In [29]:

      # Process each row
image_id_map = {}
annotation_id = 1
for _, row in df.iterrows():
    img_filename = row["fname"]
    if img_filename not in image_id_map:
        image_id = len(image_id_map) + 1
        image_id_map[img_filename] = image_id
        coco_data["images"].append({"id": image_id, "file_name": img_filename})
    
    coco_data["annotations"].append({
        "id": annotation_id,
        "image_id": image_id_map[img_filename],
        "category_id": category_mapping[row["structure"]],
        "bbox": [row["w_min"], row["h_min"], row["w_max"] - row["w_min"], row["h_max"] - row["h_min"]],
        "iscrowd": 0
    })
    annotation_id += 1


In [30]:
# Save to JSON
coco_annotations_path = "annotations_coco.json"
with open(coco_annotations_path, "w") as f:
    json.dump(coco_data, f)

print(f"COCO annotations saved to {coco_annotations_path}!")

COCO annotations saved to annotations_coco.json!


In [18]:
import cv2
import os
import glob
from sklearn.model_selection import train_test_split


In [19]:
# Load image paths (Update path to reflect actual dataset location)
image_dir = "/home/yukta/devel/wip/AIML/ultrasound-analysis/data/images"
image_paths = glob.glob(os.path.join(image_dir, "**", "*.png"), recursive=True)  # Recursive search for all PNG images

# Split dataset into training and testing sets
train_imgs, test_imgs = train_test_split(image_paths, test_size=0.2, random_state=42)

print(f"Training images: {len(train_imgs)}, Testing images: {len(test_imgs)}")


Training images: 1347, Testing images: 337


In [23]:
from ultralytics import YOLO

In [24]:
yolo_model = YOLO("yolov8n.pt")  # Load pre-trained YOLOv8 nano model

In [25]:
# Train the model
yolo_model.train(data="/home/yukta/devel/wip/AIML/ultrasound-analysis/data/dataset.yaml", epochs=50, imgsz=640)

Ultralytics 8.3.101 🚀 Python-3.13.2 torch-2.6.0+cu124 CPU (AMD Ryzen 7 5700U with Radeon Graphics)
engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=/home/yukta/devel/wip/AIML/ultrasound-analysis/data/dataset.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train11, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save

train: Scanning /home/yukta/devel/wip/AIML/ultrasound-analysis/data/labels/train

train: New cache created: /home/yukta/devel/wip/AIML/ultrasound-analysis/data/labels/train.cache



val: Scanning /home/yukta/devel/wip/AIML/ultrasound-analysis/data/labels/train.c


Plotting labels to runs/detect/train11/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.000769, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
TensorBoard: model graph visualization added ✅
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train11
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


...

                   all        812        598      0.934      0.929      0.976      0.833

50 epochs completed in 3.921 hours.


Optimizer stripped from runs/detect/train11/weights/last.pt, 6.3MB
Optimizer stripped from runs/detect/train11/weights/best.pt, 6.3MB

Validating runs/detect/train11/weights/best.pt...
Ultralytics 8.3.101 🚀 Python-3.13.2 torch-2.6.0+cu124 CPU (AMD Ryzen 7 5700U with Radeon Graphics)
Model summary (fused): 72 layers, 3,007,403 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 26/26 [00:49<00:00,  1.91s/it]


                   all        812        598      0.934      0.929      0.976      0.833
               thalami        173        173      0.925      0.923      0.972      0.851
              midbrain        214        214      0.932      0.902      0.975      0.799
                palate        211        211      0.944      0.962      0.981      0.851
Speed: 1.6ms preprocess, 51.8ms inference, 0.0ms loss, 0.3ms postprocess per image
Results saved to runs/detect/train11


ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0, 1, 2])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x7f54f2e4ec80>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.04

In [ ]:
# Ensure annotations exist
if not os.path.exists(coco_annotations_path):
    raise FileNotFoundError(f"Annotations file {coco_annotations_path} does not exist!")


In [ ]:
# Train model on ultrasound data
yolo train data=dataset.yaml model=yolov8n.pt epochs=50 imgsz=640


In [ ]:
# Perform inference
sample_image = "sample_ultrasound.png"  # Ensure this image exists
if not os.path.exists(sample_image):
    raise FileNotFoundError(f"Sample image {sample_image} not found!")

results = yolo_model(sample_image)

# Visualize results
results.show()

In [ ]:
coco_data = {"images": [], "annotations": [], "categories": []}


In [ ]:
with open("annotations_coco.json", "w") as f:
    json.dump(coco_data, f)

print("COCO annotations saved!")

In [ ]:
# Perform inference
sample_image = "/home/yukta/devel/wip/AIML/ultrasound-analysis/data/images/sample_ultrasound.png"  # Update path if needed
if not os.path.exists(sample_image):
    raise FileNotFoundError(f"Sample image {sample_image} not found!")

results = yolo_model(sample_image)

# Visualize results
results.show()